# DBSCAN

## Introduction
DBSCAN(Density-Based Spatial Clustering of Applications with Noise) is a clustering algorithm.  
The data are clustered based on the density of the data points.


## Definition
- Epsilon: The distance that defines the nearby points. If the distance between two points is less than epsilon, then they are nearby points.
- Min Points: The minimum number of points to form a cluster.
- Core Point: A point is a core point if there are at least min Points nearby within the distance epsilon.
- Border Point: A point is a border point if there are less than min Points nearby within the distance epsilon, but the point is reachable from a core point.
- Noise Point: A point is a noise point that is neither a core point nor a border point.
- Cluster: A cluster is a set of core points and border points that are density-reachable from the core points.

## Algorithm
1. Set the Hyperparameters: epsilon and min Points. 
2. Randomly select a point P.  
3. Find all the points nearby P within the distance epsilon.
4. If the number of points nearby P not less than min Points, then P is a core point.
5. All the points nearby P are in the same cluster. Merge the clusters if they have common points. 
6. Repeat the process 2 to 5 until all the points are visited.  

## Hyperparameters
1. epsilon: The distance to find the nearby points.
2. min Points: The minimum number of points to form a cluster.

## Implementation

In [ ]:
import numpy as np

class DBSCAN:
    def __init__(self, epsilon:float, minPoints:int):
        self.epsilon = epsilon
        self.minPoints = minPoints
    
    def fit(self, X:np.ndarray):
        n, m = X.shape
        self.labels = np.full(n, -1)
        self.cluster_cnt = 0
        
        for i in range(n):
            if self.labels[i] != -1:
                continue
            neighbours = self._find_neighbours(X, i)
            if len(neighbours) >= self.minPoints:
                self.labels[i] = 0
                continue
            self.cluster_cnt += 1
            self.labels[i] = self.cluster_cnt
            for j in neighbours:
                if self.labels[j] == -1:
                    self.labels[j] = self.cluster_cnt
                if self.labels[j] == 0:
                    self.labels[j] = self.cluster_cnt
                    neighbours = np.union1d(neighbours, self._find_neighbours(X, j))
            
    def _find_neighbours(self, X:np.ndarray, i:int):
        return np.linalg.norm(X - X[i], axis=1) < self.epsilon
                